# 下载Google earth data

In [4]:
import ee
import geemap
import os
import math
import geopandas as gpd
# 用户验证
# ee.Authenticate()
# 初始化Google Earth Engine
ee.Initialize()

In [ ]:
# 切分区域
def split_region(region, col_size, row_size):
    bounds = region.getInfo()['coordinates'][0]
    original_minx, original_miny = bounds[0]
    original_maxx, original_maxy = bounds[2]
    
    x_len = original_maxx - original_minx
    y_len = original_maxy - original_miny
    
    # 向上取整
    cols = math.ceil(x_len / col_size)
    rows = math.ceil(y_len / row_size)
    
    rectangles = []
    for i in range(cols):
        current_minx = original_minx + i * col_size
        current_maxx = current_minx + col_size
        if current_maxx > original_maxx:
            current_maxx = original_maxx
        for j in range(rows):
            current_miny = original_miny + j * row_size
            current_maxy = current_miny + row_size
            if current_maxy > original_maxy:
                current_maxy = original_maxy
            rect = ee.Geometry.Rectangle([current_minx, current_miny, current_maxx, current_maxy])
            rectangles.append(rect)
    return rectangles

# 栅格过滤函数

In [ ]:
# 筛选无云区域QA60
def mask_s2_clouds_QA(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)
# 筛选无云区域SCL
def mask_s2_clouds_SCL(image):
    """Masks clouds in a Sentinel-2 image using the SCL band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    scl = image.select('SCL')

    # Define the cloud mask. Typically, values 3, 8, 9, 10, and 11 in the SCL
    # band represent cloud and shadow conditions.
    cloud_shadow = scl.eq(3)  # Cloud shadows
    cloud_medium_prob = scl.eq(8)  # Medium probability clouds
    cloud_high_prob = scl.eq(9)  # High probability clouds
    thin_cirrus = scl.eq(10)  # Thin cirrus
    cloud_mask = cloud_shadow.Or(cloud_medium_prob).Or(cloud_high_prob).Or(thin_cirrus)

    return image.updateMask(cloud_mask.Not()).divide(10000)

In [ ]:
# 读取本地Shapefile 得到处理区域
shapefile_path = r"F:\ArcgisData\shp_polygon\KY500.shp"
gdf = gpd.read_file(shapefile_path)


bounds = gdf['geometry'][0].bounds
bbox = {
    'MinX': bounds[0],
    'MinY': bounds[1],
    'MaxX': bounds[2],
    'MaxY': bounds[3]
}

# 获取第一个几何对象
geometry = gdf.geometry.iloc[0]
# 2. 将 Earth Engine 几何对象转换为 Feature 对象
ee_geometry = ee.Geometry(geometry.__geo_interface__)
# 3. 将 Earth Engine 几何对象转换为 Feature 对象
feature = ee.Feature(ee_geometry)

# 4. 将 Feature 对象放入 FeatureCollection 中
feature_collection = ee.FeatureCollection([feature])

# 获取 FeatureCollection 的边界几何对象
bounded_geometry = feature_collection.geometry().bounds()
# 矢量边框
vector_layer = ee.FeatureCollection([ee.Feature(ee_geometry)])

# 在地图中查看筛选结果

In [ ]:
# 时间范围
start_date = '2023-10-01'  # 开始日期
end_date = '2023-12-30'    # 结束日期
# 云概率
cloud_probability_threshold = 80  # 云概率阈值
# 函数
functions = mask_s2_clouds_SCL

In [ ]:
# 创建 Sentinel-2 数据集并过滤指定区域
view_dataset = (
    # ee.ImageCollection('COPERNICUS/S2_SR')
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_probability_threshold))
    .filterBounds(ee_geometry)
    .map(functions)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# 创建地图对象并添加图层控制
m = geemap.Map()
m.addLayerControl()
# 添加 Sentinel-2 数据图层
m.addLayer(view_dataset.median(), visualization, 'RGB')
# 添加矢量范围的边框
m.addLayer(vector_layer.style(**{'color': 'red', 'fillColor': '00000000'}), {}, 'Vector Boundary')
# 自动缩放到矢量图层
m.centerObject(vector_layer, zoom=12)
# 显示地图
m

# 下载数据

In [ ]:
# 定义要下载的图像的参数
area_of_interest =ee.Geometry.Rectangle([bbox['MinX'], bbox['MinY'], bbox['MaxX'], bbox['MaxY']])  # 请替换经度和纬度

bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12','QA60','SCL']  # 选择要下载的波段

# 创建Sentinel-2图像集合
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(area_of_interest) \
    .select(bands) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_probability_threshold)) \
    .map(functions)

In [ ]:
polygon_list = split_region(bounded_geometry,0.05,0.05)

In [ ]:
len(polygon_list)

In [ ]:
# 设置输出文件路径
output_folder = r"F:\GEEDOWNLOAD\sentinel2\KY_20231001_20231230"  # 输出文件夹路径
output_file_prefix = 'Sentinel-2_scl'
idx=1
for one_polygon in polygon_list:
    file_name = os.path.join(output_folder,f"{output_file_prefix}_{idx}.tif")
    if os.path.exists(file_name):
        print('已存在')
    else:
        geemap.ee_export_image(dataset.median(), filename=file_name, region=one_polygon,scale=10)
    idx+=1
    print(idx)